In [45]:
using LinearAlgebra
using ExponentialUtilities

include("../test_data.jl")
include("../graph_utils.jl")

# using Lux
# using Random
# using Zygote
# using Optimisers
# using Plots
# using Distributions
# using Statistics

global_efficiency (generic function with 1 method)

In [91]:
ExpMethodHigham2005(W_Y)

ExpMethodHigham2005(true)

In [70]:
W_Y, D, A_init, coord = load_weight_test_data()
global_efficiency(W_Y)

W_init = rand(size(W_Y)...)
W_init = W_init .* (W_Y .> 0);

In [92]:
ω = 1

function loss(W)
    # compute S
    node_strengths = dropdims(sum(W, dims=2), dims=2)
    node_strengths[node_strengths.==0] .= 1e-5
    S = sqrt(inv(Diagonal(node_strengths)))

    # compute communicability
    #method=ExpMethodHigham2005();
    #cache=ExponentialUtilities.alloc_mem(W,method);
    #C = exponential( S * W * S, method, cache)

    C = exp(S*W*S)
    C[diagind(C)] .= 0.0

    optimal_eficiency =  global_efficiency((1/maximum(W_Y) .+ zeros(size(W_Y))) .* (W_Y .> 0))
    current_efficiency = global_efficiency(W) 
    communicability = sum((W .* C .* D) .^ ω)

    # the weights must be at that position nad larger than zero
    return (optimal_eficiency - current_efficiency) + communicability
end

loss (generic function with 2 methods)

In [48]:
loss(W_Y, 0.1)

512.7366084367804

In [78]:
function gradient_descent(f, x, η, T; callback = x -> nothing)
    for t in 1:T
		∇f = gradient(f, x)[1] # compute the gradient
        println("∇f: ", ∇f)
        x .-= η * ∇f # update parameters in direction of -∇f
        callback(x) # the callback will be used to save intermediate values
    end
    x
end

gradient_descent (generic function with 1 method)

In [76]:
function example()
    Random.seed!(5)
    f(x, θ) = θ[1] * sin.(θ[2] * x .+ θ[3]) .+ θ[4] * sin.(θ[5] * x .+ θ[6])
    f(x) = 0.3 * sin(2x) + .7 * sin(3.5x + 1)
    x3 = randn(50)
    y3 = f.(x3)
    mse(θ) = mean((y3 .- f(x3, θ)).^2)
    gradient_descent(mse, params3, .1, 10^4)
end



example (generic function with 1 method)

In [93]:
gradient_descent(loss, W_init, .1, 10^4)

LoadError: Mutating arrays is not supported -- called copyto!(SubArray{Float64, 1, Vector{Float64}, Tuple{StepRange{Int64, Int64}}, true}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [82]:
alloc_mem

LoadError: UndefVarError: `alloc_mem` not defined